In [1]:
import os
import cv2
import numpy as np
import tensorflow as tf
import sys
import time
import pandas as pd

from tqdm import tqdm
# Import utilites
from object_detection.utils import label_map_util
from object_detection.utils import visualization_utils as vis_util

In [2]:
def output_results(IMAGE_NAME):
    # Path to frozen detection graph .pb file, which contains the model that is used
    # for object detection.
    PATH_TO_CKPT = os.path.join(CWD_PATH,'inference','frozen_inference_graph.pb')

    # Path to label map file
    PATH_TO_LABELS = os.path.join(CWD_PATH,'training','labelmap.pbtxt')

    # Path to image
    PATH_TO_IMAGE = os.path.join(CWD_PATH,'Testing',IMAGE_NAME)

    # Number of classes the object detector can identify
    NUM_CLASSES = 1# Load the label map.
    # Label maps map indices to category names, so that when our convolution
    # network predicts `5`, we know that this corresponds to `king`.
    # Here we use internal utility functions, but anything that returns a
    # dictionary mapping integers to appropriate string labels would be fine
    label_map = label_map_util.load_labelmap(PATH_TO_LABELS)
    categories = label_map_util.convert_label_map_to_categories(label_map, max_num_classes=NUM_CLASSES, use_display_name=True)
    category_index = label_map_util.create_category_index(categories)
    
    # Load the Tensorflow model into memory.
    detection_graph = tf.Graph()
    with detection_graph.as_default():
        od_graph_def = tf.GraphDef()
        with tf.gfile.GFile(PATH_TO_CKPT, 'rb') as fid:
            serialized_graph = fid.read()
            od_graph_def.ParseFromString(serialized_graph)
            tf.import_graph_def(od_graph_def, name='')

        sess = tf.Session(graph=detection_graph)
    
    # Define input and output tensors (i.e. data) for the object detection classifier
    # Input tensor is the image
    image_tensor = detection_graph.get_tensor_by_name('image_tensor:0')

    # Output tensors are the detection boxes, scores, and classes
    # Each box represents a part of the image where a particular object was detected
    detection_boxes = detection_graph.get_tensor_by_name('detection_boxes:0')

    # Each score represents level of confidence for each of the objects.
    # The score is shown on the result image, together with the class label.
    detection_scores = detection_graph.get_tensor_by_name('detection_scores:0')
    detection_classes = detection_graph.get_tensor_by_name('detection_classes:0')

    # Number of objects detected
    num_detections = detection_graph.get_tensor_by_name('num_detections:0')

    # Load image using OpenCV and
    # expand image dimensions to have shape: [1, None, None, 3]
    # i.e. a single-column array, where each item in the column has the pixel RGB value
    image = cv2.imread(PATH_TO_IMAGE)
    image_expanded = np.expand_dims(image, axis=0)
    
    # Perform the actual detection by running the model with the image as input
    (boxes, scores, classes, num) = sess.run(
        [detection_boxes, detection_scores, detection_classes, num_detections],
        feed_dict={image_tensor: image_expanded})
    # Draw the results of the detection (aka 'visulaize the results')
    vis_util.visualize_boxes_and_labels_on_image_array(
        image,
        np.squeeze(boxes),
        np.squeeze(classes).astype(np.int32),
        np.squeeze(scores),
        category_index,
        use_normalized_coordinates=True,
        line_thickness=8,
        min_score_thresh=0.60)
    
    # All the results have been drawn on image. Now display the image.
    SAVEFILE = 'Result' + IMAGE_NAME
    SAVEDIRECTORY = str(os.path.join(CWD_PATH,'Results', SAVEFILE))
    cv2.imwrite(SAVEDIRECTORY, image)
    return 1, boxes, scores

In [7]:
# Grab path to current working directory
CWD_PATH = os.getcwd()
images = os.path.join(CWD_PATH,'Testing')
data = pd.DataFrame(columns=['IMAGE', 'Y_min', 'X_min','Y_max','X_max'])
for filename in tqdm(os.listdir(images)):
    x, boxes, scores = output_results(filename)
    count = 0
    for i in scores[0]:
        if i > 0.8:
            count+=1
        else:
            break
    for i in range(count):
        data = data.append({'IMAGE':str(filename.split('.')[0]), 'Y_min':boxes[0][i][0], 'X_min':boxes[0][i][1],
                     'Y_max':boxes[0][i][2],'X_max':boxes[0][i][3]}, ignore_index=True)


 21%|█████████████████▏                                                               | 67/316 [11:21<41:07,  9.91s/it]


 43%|██████████████████████████████████▏                                             | 135/316 [21:14<24:01,  7.97s/it]


 64%|███████████████████████████████████████████████████▍                            | 203/316 [30:14<14:54,  7.92s/it]


 86%|████████████████████████████████████████████████████████████████████▌           | 271/316 [39:08<05:55,  7.90s/it]


100%|████████████████████████████████████████████████████████████████████████████████| 316/316 [46:08<00:00,  9.39s/it]

In [8]:
data.head()

,IMAGE,Y_min,X_min,Y_max,X_max
0,GOPR0388_0,0.323994,0.324396,0.480392,0.415359
1,GOPR0388_10,0.336630,0.229216,0.637490,0.306214
2,GOPR0388_10,0.301154,0.820965,0.605084,0.873851
3,GOPR0388_10,0.007562,0.869734,0.394208,0.991250
4,GOPR0388_10,0.129536,0.875168,0.424447,0.927029


In [9]:
data.to_csv('Boundingbox_Coordinates.csv')

In [72]:
#creating new dataframe to scale the data proerly
data1 = pd.DataFrame(columns=['IMAGE', 'Y_min', 'X_min','Y_max','X_max'])
data1['IMAGE'] = data['IMAGE'].str.split('_',n=1,expand=True)[0]
#Scaling data properly
data1['Y_min'] = data['Y_min'] * 1080
data1['X_min'] = data['X_min'] * 1920
data1['Y_max'] = data['Y_max'] * 1080
data1['X_max'] = data['X_max'] * 1920

#Converting to int
data1.Y_min = data1.Y_min.round().astype(int)
data1.X_min = data1.X_min.round().astype(int)
data1.Y_max = data1.Y_max.round().astype(int)
data1.X_max = data1.X_max.round().astype(int)

data1.head()

,IMAGE,Y_min,X_min,Y_max,X_max
0,GOPR0388,350,623,519,797
1,GOPR0388,364,440,688,588
2,GOPR0388,325,1576,653,1678
3,GOPR0388,8,1670,426,1903
4,GOPR0388,140,1680,458,1780


In [78]:
height_label = pd.read_excel('video_labels.xlsx')
height_label.head()

,4'Video,2'Video,EarHt (inches)
0,GOPR0388,GOPR0165,39.0
1,GOPR0389,GOPR0166,50.0
2,GOPR0390,GOPR0167,44.0
3,GOPR0391,GOPR0168,40.0
4,GOPR0392,GOPR0169,40.0


In [81]:
height_label = height_label.drop(columns='2\'Video')
height_label.head()

,4'Video,EarHt (inches)
0,GOPR0388,39.0
1,GOPR0389,50.0
2,GOPR0390,44.0
3,GOPR0391,40.0
4,GOPR0392,40.0


In [96]:
earHt = []
for i in tqdm(range(len(data1))):
    #getting the index in height label dataframe where the video number matches to work with
    index = height_label.index[height_label["4'Video"]==data1.iloc[i][0]]
    #getting the whole row data where index matche
    value = height_label.iloc[index]
    #getting the height and appending it to the array
    height = value.iloc[0]['EarHt (inches)']
    earHt.append(height)
    
data1['EarHt (inches)'] = earHt


100%|███████████████████████████████████████████████████████████████████████████████| 553/553 [00:01<00:00, 318.85it/s]

In [97]:
data1.to_csv('Boundingbox_Coordinates.csv', index=False)